In [1]:
import numpy as np
import pandas as pd
import os
from inda_mir.loaders import load_model, load_data_partition
from settings import settings
from tqdm import tqdm

In [2]:
misd_path = '/mnt/y/CODES/track_classifier/misd'

In [3]:
train_metadata = pd.read_csv(os.path.join(misd_path, 'Metadata_Train.csv'))
test_metadata = pd.read_csv(os.path.join(misd_path, 'Metadata_Test.csv'))
train_path = os.path.join(misd_path, 'Train_submission/Train_submission')
test_path = os.path.join(misd_path, 'Test_submission/Test_submission')

In [4]:
train_metadata['Class'].value_counts()

Class
Sound_Guitar    700
Sound_Drum      700
Sound_Violin    700
Sound_Piano     529
Name: count, dtype: int64

In [5]:
import os

def rename_files_to_lowercase(directory):
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            # Get the lowercase version of the filename
            lowercase_filename = filename.lower()
            
            # Get the path to the lowercase filename
            lowercase_filepath = os.path.join(directory, lowercase_filename)
            
            # Rename the file to the lowercase version
            os.rename(os.path.join(directory, filename), lowercase_filepath)


# rename_files_to_lowercase(train_path)
# rename_files_to_lowercase(test_path)
# train_metadata['FileName'] = train_metadata['FileName'].apply(lambda x: x.lower())
# test_metadata['FileName'] = test_metadata['FileName'].apply(lambda x: x.lower())

In [6]:
train_metadata['FileName'] = train_metadata['FileName'].apply(lambda x: x.replace(" ", ""))
train_metadata['FileName'] = train_metadata['FileName'].apply(lambda x: x.replace("-", "_"))

In [7]:
train_metadata['path'] = train_metadata['FileName'].apply(lambda x: os.path.join(train_path, x))
test_metadata['path'] = test_metadata['FileName'].apply(lambda x: os.path.join(test_path, x))
train_metadata['prediction'] = 0
test_metadata['prediction'] = 0

In [8]:
train_metadata.head()

,FileName,Class,path,prediction
0,1_E1_Major00.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/Train_submi...,0
1,1_E1_Major01.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/Train_submi...,0
2,1_E1_Major02.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/Train_submi...,0
3,1_E1_Major03.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/Train_submi...,0
4,1_E1_Major04.wav,Sound_Guitar,/mnt/y/CODES/track_classifier/misd/Train_submi...,0


In [9]:
train_metadata['path'].iloc[0]

'/mnt/y/CODES/track_classifier/misd/Train_submission/Train_submission/1_E1_Major00.wav'

In [10]:
model = load_model(os.path.join(settings.MODEL_OUTPUT_PATH, settings.MODEL_NAME))
model

In [11]:
model.get_n_features()

253

In [12]:
# from inda_mir.modeling.models.lgbm import LightGBMClassifier

# features = '/mnt/y/CODES/track_classifier/output/features/inda_mir_features_without_brass_v2.csv'
# model = LightGBMClassifier()

# model.fit(features)

In [13]:
from inda_mir.modeling.models.classifier import InstrumentClassifier


classifier = InstrumentClassifier(model)

print(os.listdir('/mnt/y/CODES/track_classifier/misd/Train_submission/Train_submission/'))
print(os.path.exists(os.path.join(train_path, 'wavedrum02_45sd_(94).wav')))

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


['029500_morning-rain-piano-65875.wav', '035568_upright-piano-71570.wav', '04-47367.wav', '046578_tribal-drum-rhythms-03wav-68096.wav', '054399_8039s-old-school-rap-drum-loop-80433.wav', '056514_drum-40118.wav', '059960_sonido2-sincopa-alta-7mp3-47316.wav', '060311_rubbing-drum-sticksaif-82905.wav', '065339_metal-bass-drum-90850.wav', '066166_qui-c39est-qu39est-tombe-loop-t85wav-39366.wav', '068320_hang-drum-2wav-80568.wav', '0_emil-telmanyi_bwv1001.wav', '0_emil-telmanyi_bwv1002.wav', '0_emil-telmanyi_bwv1003.wav', '0_emil-telmanyi_bwv1004.wav', '0_emil-telmanyi_bwv1005.wav', '0_emil-telmanyi_bwv1006.wav', '0_john-garner_bwv1002_mov1.wav', '0_john-garner_bwv1002_mov2.wav', '0_john-garner_bwv1002_mov3.wav', '0_john-garner_bwv1002_mov4.wav', '0_john-garner_bwv1002_mov5.wav', '0_john-garner_bwv1002_mov6.wav', '0_john-garner_bwv1002_mov7.wav', '0_john-garner_bwv1002_mov8.wav', '0_kinga-augustyn_bwv1005_mov2.wav', '0_ko-donghwi_bwv1001.wav', '0_minji-kim_bwv1003_mov1.wav', '0_minji-kim_bwv

In [28]:
# test_metadata['prediction'] = test_metadata['path'].apply(lambda x: classifier.classify(x))
csv = os.path.join(misd_path, 'test_predictions.csv')
if os.path.exists(csv):
    test_metadata = pd.read_csv(csv)

for i, row in tqdm(test_metadata[test_metadata['prediction'] == '0'].iterrows()):
    # if row['prediction'] != 0:
    #     continue
    print(row['path'])
    pred = classifier.classify(row['path']).instrument
    test_metadata.loc[i, 'prediction'] = pred
    test_metadata.to_csv(os.path.join(misd_path, 'test_predictions.csv'))
    os.system('cls' if os.name == 'nt' else 'clear')

0it [00:00, ?it/s]2024-04-17 04:38:30,927 - INFO - classifier - Extracting /mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-A-wwwfesliyanstudioscom features...


/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-A-www.fesliyanstudios.com.wav
/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-A-www.fesliyanstudios.com.wav
['Sad-Violin-A-www', 'fesliyanstudios', 'com', 'wav']


2024-04-17 04:38:31,162 - INFO - classifier - Classified "/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-A-wwwfesliyanstudioscom" as "silence"!
1it [00:00,  3.93it/s]2024-04-17 04:38:31,185 - INFO - classifier - Extracting /mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-C-wwwfesliyanstudioscom features...


/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-C-www.fesliyanstudios.com.wav
/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-C-www.fesliyanstudios.com.wav
['Sad-Violin-C-www', 'fesliyanstudios', 'com', 'wav']


2024-04-17 04:38:31,490 - INFO - classifier - Classified "/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-C-wwwfesliyanstudioscom" as "silence"!
2it [00:00,  3.40it/s]2024-04-17 04:38:31,503 - INFO - classifier - Extracting /mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/Sad-Violin-D-wwwfesliyanstudioscom features...


...

2024-04-17 04:38:45,173 - INFO - classifier - Classified "/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/whatever-you-say-piano-27291" as "silence"!
37it [00:14,  2.43it/s]2024-04-17 04:38:45,187 - INFO - classifier - Extracting /mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wingrandpiano-96338 features...
2024-04-17 04:38:45,299 - INFO - classifier - Classified "/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wingrandpiano-96338" as "silence"!
38it [00:14,  3.06it/s]2024-04-17 04:38:45,314 - INFO - classifier - Extracting /mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wondercue-111933 features...


/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wingrandpiano-96338.wav
/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wingrandpiano-96338.wav
['wingrandpiano-96338', 'wav']
/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wondercue-111933.wav
/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wondercue-111933.wav
['wondercue-111933', 'wav']


2024-04-17 04:38:46,006 - INFO - classifier - Classified "/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/wondercue-111933" as "silence"!
39it [00:15,  2.27it/s]2024-04-17 04:38:46,021 - INFO - classifier - Extracting /mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/yuwu-quiz-113046 features...


/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/yuwu-quiz-113046.wav
/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/yuwu-quiz-113046.wav
['yuwu-quiz-113046', 'wav']


2024-04-17 04:38:46,451 - INFO - classifier - Classified "/mnt/y/CODES/track_classifier/misd/Test_submission/Test_submission/yuwu-quiz-113046" as "silence"!
40it [00:15,  2.57it/s]

In [27]:
test_metadata.prediction.value_counts()

prediction
0            40
guitar       18
drums        16
other         4
keyboards     1
silence       1
Name: count, dtype: int64